### Unit 1 Assignment

Train an agent for the Lunar-Landerv2 environment using Stable-Baselines3 library

In [3]:
import os
os.environ['PATH'] += '/common/home/skh79/local/xvfb/bin/Xvfb'

In [4]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

FileNotFoundError: [Errno 2] No such file or directory: 'Xvfb'

In [1]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [2]:
import gymnasium as gym

# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v2")

# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken:", action)

  # Do this action in the environment and get
  # next_state, reward, terminated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)

  # If the game is terminated (in our case we land, crashed) or truncated (timeout)
  if terminated or truncated:
      # Reset the environment
      print("Environment is reset")
      observation, info = env.reset()

env.close()

Action taken: 0
Action taken: 2
Action taken: 3
Action taken: 0
Action taken: 0
Action taken: 0
Action taken: 0
Action taken: 0
Action taken: 2
Action taken: 1
Action taken: 1
Action taken: 1
Action taken: 0
Action taken: 3
Action taken: 1
Action taken: 3
Action taken: 2
Action taken: 1
Action taken: 2
Action taken: 0


In [3]:
# We create our environment with gym.make("<name_of_the_environment>")
env = gym.make("LunarLander-v2")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [ 4.2897957e+01 -1.6988243e+01  3.0034523e+00  2.1834114e+00
 -1.6388742e+00  3.7806518e+00  6.5736181e-01  2.0748030e-02]


We see with Observation Space Shape (8,) that the observation is a vector of size 8, where each value contains different information about the lander:
-  Horizontal pad coordinate (x)
- Vertical pad coordinate (y)
- Horizontal speed (x)
- Vertical speed (y)
- Angle
- Angular speed
- If the left leg contact point has touched the land (boolean)
- If the right leg contact point has touched the land (boolean)

In [4]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 3


The action space (the set of possible actions the agent can take) is discrete with 4 actions available 🎮:

- Action 0: Do nothing,
- Action 1: Fire left orientation engine,
- Action 2: Fire the main engine,
- Action 3: Fire right orientation engine.

Reward function (the function that will give a reward at each timestep) 💰:

After every step a reward is granted. The total reward of an episode is the **sum of the rewards for all the steps within that episode**.

For each step, the reward:

- Is increased/decreased the closer/further the lander is to the landing pad.
-  Is increased/decreased the slower/faster the lander is moving.
- Is decreased the more the lander is tilted (angle not horizontal).
- Is increased by 10 points for each leg that is in contact with the ground.
- Is decreased by 0.03 points each frame a side engine is firing.
- Is decreased by 0.3 points each frame the main engine is firing.

The episode receive an **additional reward of -100 or +100 points for crashing or landing safely respectively.**

An episode is **considered a solution if it scores at least 200 points.**

#### Vectorized Environment

- We create a vectorized environment (a method for stacking multiple independent environments into a single environment) of 16 environments, this way, **we'll have more diverse experiences during the training.**

In [5]:
# Create the environment
env = make_vec_env('LunarLander-v2', n_envs=16)

In [6]:
?PPO

Init signature:
PPO(
    policy: Union[str, Type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gymnasium.core.Env, stable_baselines3.common.vec_env.base_vec_env.VecEnv, str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    normalize_advantage: bool = True,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    target_kl: Optional[float] = None,
    stats_window_size: int = 100,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[Dict[str, Any]] = None,
    verbose: int = 0,
    seed: Optional[int] = None,
    device: Union[torch.device, str] = 'auto',
    _init_setup

In [7]:
import torch
torch.cuda.is_available()

True

In [8]:
# Instantiate MlpPolicy model so that it works with vectorized environment env
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.9995,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)

Using cuda device


In [9]:
model_name = 'ppo-Lunar-Lander-v2'

# do a small amount of training and run it to see how well we do
model.learn(total_timesteps=500000);

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 96.2     |
|    ep_rew_mean     | -204     |
| time/              |          |
|    fps             | 2876     |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 16384    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 94           |
|    ep_rew_mean          | -173         |
| time/                   |              |
|    fps                  | 1650         |
|    iterations           | 2            |
|    time_elapsed         | 19           |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.008593319  |
|    clip_fraction        | 0.034        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | -0.000518322 |
|    learning_r

In [8]:
# Evaluating our model in a new, not vectorized environment

# We wrap the environment in a monitor so it can be displayed
eval_env = Monitor(gym.make("LunarLander-v2", render_mode='rgb_array'))

In [10]:
# Stable-Baselines3 provides function evaluate_policy to check how well our model performs
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=-12.38 +/- 21.713644556799206


In [9]:
model = PPO.load('models/ppo-lander-curr.zip')

In [11]:
# We can compare by loading our previous model:
from huggingface_sb3 import load_from_hub
checkpoint = load_from_hub(
	repo_id="shazeghi/ppo-LunarLander-v2",
	filename="ppo-LunarLander-v2.zip",
)

In [24]:
prev_model = PPO.load(checkpoint, env=env)
prev_model

In [14]:
def evaluate(model, env=eval_env):
    mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
    print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

evaluate(model)
evaluate(prev_model)

    

mean_reward=-19.75 +/- 17.764647946642366
mean_reward=224.81 +/- 32.502890864698415


In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

# Assuming you have a vectorized environment setup
vec_env = make_vec_env('LunarLander-v2', n_envs=16)

prev_model.learn(total_timesteps=3000000)

prev_model.save('models/ppo-lander-3m')

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 306      |
|    ep_rew_mean     | 227      |
| time/              |          |
|    fps             | 1817     |
|    iterations      | 1        |
|    time_elapsed    | 9        |
|    total_timesteps | 16384    |
---------------------------------
